In [0]:
print("Healthy")

In [0]:
from re import search
from sqlalchemy import create_engine
import duckdb

import pandas as pd 
import numpy as np 
import mysql.connector
import pymysql


pd.set_option('display.max_columns', None)


import geopy
from geopy.geocoders import ArcGIS, Nominatim

import warnings
warnings.filterwarnings('ignore')

import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


import pymysql
mifos_conn = pymysql.connect(user = data['mifos']['user'], 
                           password = data['mifos']['passwd'], 
                           database = data['mifos']['database'], 
                           host = data['mifos']['host'])
mifos_cursor = mifos_conn.cursor()


#mifos_engine = create_engine(f"mysql+pymysql://{data['mifos']['user']}:{data['mifos']['passwd']}@{data['mifos']['host']}:3306/{data['mifos']['database']}")
#mifos_connection = mifos_engine.connect()


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)

host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")

pd.set_option('display.float_format', lambda x: '%.2f' % x)

from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d')
print(today)
print(yesterday)



In [0]:
%%time
## Client Image_id table to solely distinguish App registered customers

cl_img = pd.read_sql(f'''
            select
            mc.id as client_id,
            mc.activation_date,
            ifnull(mc.activatedon_userid, 0) as activatedon_userid,
            mc.submittedon_userid,
            ifnull(mc.image_id, 0) as image_id,
            case when image_id is not null then 1 else 0 end as is_appclient
            from m_client mc
            ''', Mifosdb)

# identify wallet clients
amw = pd.read_sql(f'''
        select clientid  as client_id from AM_WALLET_CLIENTS
        ''', Dumpdb)

amw['client_id'] = amw['client_id'].astype('int64')

# identify corporate clients
corp = pd.read_sql(f'''
                    SELECT
                    mc.id AS client_id
                    FROM m_client mc
                    JOIN m_savings_account sa ON sa.client_id = mc.id
                    WHERE sa.product_id = 26;
                    ''', Mifosdb)

cl_img['is_walletclient'] = np.where(cl_img['client_id'].isin(amw['client_id']), 1, 0)
cl_img['is_corpclient'] = np.where(cl_img['client_id'].isin(corp['client_id']), 1, 0)


# differentiating appclients aka retail customers from wallet customers and corporate customers to be used later --
## for classification on the main client table
appclient = cl_img[cl_img['is_appclient'] == 1]['client_id'].tolist()
wallclient = cl_img[cl_img['is_walletclient'] == 1]['client_id'].tolist()
corpclient = cl_img[cl_img['is_corpclient'] == 1]['client_id'].tolist()

cl_img.head()

In [0]:
%%time

cl_img.to_sql("dwh_client_imageid", conn, index = False, if_exists  = 'replace', chunksize = 10000, method = 'multi') 

## Standard individual clients

In [0]:
clt = pd.read_sql_query(f'''
        select
        max(client_id) as client
        from dwh_all_clients
        ''', conn)

last_clt = clt['client'][0]

print(last_clt)

In [0]:
import pymysql
mifos_conn = pymysql.connect(user = data['mifos']['user'], 
                           password = data['mifos']['passwd'], 
                           database = data['mifos']['database'], 
                           host = data['mifos']['host'])
mifos_cursor = mifos_conn.cursor()

sql =f"""
select
mc.id client_id,
replace(mc.display_name, 'null', '') as client_name,
mc.firstname, 
mc.middlename, 
mc.lastname,
case mc.status_enum 
when '0' then 'Invalid'
when '100' then 'Pending'
when '300' then 'Active'
when '600' then 'Closed' end as 'client_status',
mc.activation_date,
mc.external_id,
ai.`Mobile Number` as mobile_number,
ai.`Email Address` as email_address,
ai.`Mailing Address` as mailing_address,
ai.`Local Govt AREA` as LGA,
a.state,
b.country,
case mc.gender_cv_id
when '29' then 'Male'
when '30' then 'Female' end as 'client_gender',
mc.date_of_birth,
case mc.client_type_cv_id 
when '64' then 'Personal'
when '65' then 'Official'
when '66' then 'Business' end as 'client_type',
case mc.client_classification_cv_id 
when '61' then 'Single'
when '62' then 'Married'
when '63' then 'Divorcee' end as 'marital_status',
mc.referral_id as referral_code,
mc.referred_by_id,
mc.daily_withdraw_limit,
mc.max_transaction_limit,
case mc.client_level_cv_id 
when '99' then 'Tier 1'
when '100' then 'Tier 2'
when '101' then 'Tier 3' end as client_tier,
odb.BVN,
map.username as 'activation_request_made_by',
mp.username as 'authorised_by',               
mc.closedon_date
from m_client mc
left join m_savings_account sa on sa.client_id = mc.id
left join m_savings_product sp on sp.id = sa.product_id
left join `Other Bank Details` odb on odb.client_id = mc.id
left join `Address Info` ai on ai.client_id = mc.id
left join (SELECT
        id state_code,
        code_description AS state
        FROM m_code_value
        WHERE code_id in (27)) a on a.state_code = ai.state_cd_state
left join (SELECT
        id country_code,
        code_description AS country
        FROM m_code_value
        WHERE code_id in (28)) b on b.country_code = ai.country_cd_country
left join m_appuser map on map.id = mc.submittedon_userid
left join m_appuser mp on mp.id = mc.activatedon_userid
where sa.client_id > {last_clt}
group by mc.id;
"""
mifos_cursor.execute(sql)
all_clts = pd.DataFrame(mifos_cursor.fetchall())

all_clts = all_clts.rename(columns={
    0: "client_id",
    1: "client_name",
    2: "firstname",
    3: "middlename",
    4: "lastname",
    5: "status_enum",
    6: "client_status",
    7: "activation_date",
    8: "mobile_number",
    9: "email_address",
    10: "mailing_address",
    11: "LGA",
    12: "state",
    13: "country",
    14: "client_gender",
    15: "date_of_birth",
    16: "client_type",
    17: "marital_status",
    18: "referral_code",
    19: "referred_by_id",
    20: "daily_withdraw_limit",
    21: "max_transaction_limit",
    22: "client_tier",
    23: "BVN",
    24: "activation_request_made_by",
    25: "authorised_by",
    26: "closedon_date",
})

all_clts.head()

In [0]:
all_clts['client_category'] = np.where((all_clts['client_id'].isin(appclient)), 'Retail client',
                               np.where((all_clts['client_id'].isin(wallclient)), 'Wallet client',
                                        np.where((all_clts['client_id'].isin(corpclient)), 'Corporate client', 'Unclassified')))

In [0]:
%%time
# identification number

um = pd.read_sql_query(f'''
            select
            ui.client_id,
            ui.identity_number
            from UM_IDENTITY_RECORDS ui
            ''', Dumpdb)
um['client_id'] = um['client_id'].astype('int64')

In [0]:
full = pd.merge(all_clts, um, on = 'client_id', how = 'left')
full = full.drop_duplicates(subset = 'client_id').reset_index(drop = True)

print('\n', full.shape)

In [0]:
%%time
full.to_sql('dwh_all_clients', conn, index=False, if_exists='append', chunksize= 10000, method = 'multi')

## Corporate CAC details

In [0]:
%%time

cac = pd.read_sql(f'''
SELECT 
id, client_id, incorp_no as cac_no, incorp_validity_till as cac_expiry_date
FROM m_client_non_person mnp
''', Mifosdb)
cac

In [0]:
%%time

cac.to_sql("dwh_corporate_cac_details", conn, index = False, if_exists = 'replace', chunksize = 10000, method = 'multi')

## Joint Account / Group clients 

In [0]:

# grp = pd.read_sql_query(f'''
#         select
#         max(group_client_id) as group
#         from dwh_group_clients
#         ''', conn)

# last_grp = grp['group'][0]

# last_grp

In [0]:
Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)

all_grp = pd.read_sql_query(f'''
                select
                mg.id as group_client_id,
                ifnull(mgc.client_id, 0) as linked_client_id,
                mg.external_id,
                case mg.status_enum 
                    when '0' then 'Invalid'
                    when '100' then 'Pending'
                    when '300' then 'Active'
                    when '600' then 'Closed' end as 'group_client_status',
                mg.activation_date,
                mg.office_id,
                mg.staff_id,
                mg.parent_id,
                mg.level_id,
                replace(mg.display_name, 'null', '') as group_client_name,
                mg.hierarchy,
                ifnull(mg.closure_reason_cv_id, 0) as closure_reason_id,
                case mg.closure_reason_cv_id 
                    when '121' then 'Error creating' end as group_closure_reason_description,
                ifnull(mg.submittedon_userid, 0) as submittedon_userid,
                map.username as 'activation_request_submitted_by',
                
                ifnull(mg.activatedon_userid, 0) as activatedon_userid,
                mp.username as 'activation_authorised_by', 
                
                ifnull(mg.closedon_userid, 0) as closedon_userid,
                mpp.username as 'closed_by',
                mg.submittedon_date,
                mg.closedon_date,
                mg.account_no
                
                from m_group mg
                left join m_group_client mgc on mgc.group_id = mg.id
                left join m_savings_account sa on sa.group_id = mg.id
                left join m_savings_product sp on sp.id = sa.product_id
                left join m_appuser map on map.id = mg.submittedon_userid
                left join m_appuser mp on mp.id = mg.activatedon_userid
                left join m_appuser mpp on mpp.id = mg.closedon_userid
                
                -- where sa.group_id > 
                group by sa.group_id
                ''', Mifosdb)

all_grp

In [0]:
%%time

all_grp.to_sql("dwh_group_clients", conn, index = False, if_exists = 'replace', chunksize = 10000, method = 'multi')